##Link Drive

In [2]:
#connect google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#access current folder
!ls
import os
os.chdir('/content/drive/My Drive/Project/评论情感分析')
!ls
#!git clone https://github.com/AimeeLee77/keyword_extraction

drive  sample_data
 negative-words.txt   result.gsheet
 positive-words.txt   sentiment_analysis.ipynb
'result (1).gsheet'   VR_Shopping_data_final.xlsx
 result.csv	      基于情感词典的情感分析方法.gdoc


##Load Data

In [4]:
import pandas as pd
data = pd.read_excel('VR_Shopping_data_final.xlsx', header = 0)#.astype(str).iloc[:,2:3]
data_id = data['Part.ID1']
data_needed = data[['Q30','Q31', 'Q45']]
data_needed = data_needed.dropna(axis=0, how='all')
data_needed['collect'] = [str(data_needed.iloc[i, 0])+ ' '+ str(data_needed.iloc[i, 1])+ ' '+ str(data_needed.iloc[i, 2]) for i in range(len(data_needed))]
data_id_needed = data_id.loc[data_needed.index]
data_needed['id'] = data_id_needed
print(data_needed)

                                                    Q30  ...   id
0                                                   NaN  ...    1
6     picked the products without really think about...  ...    2
18                                                  NaN  ...    4
25    smooth response to controls and movement, ease...  ...    5
53                                                  NaN  ...    7
...                                                 ...  ...  ...
2048  quite peaceful but almost a bit too quiet. Fel...  ...  146
2070                                                NaN  ...  148
2077  I was looking at all the available items to se...  ...  149
2101                                                NaN  ...  151
2114  Hoping the hand-helds would work quickly when ...  ...  152

[102 rows x 5 columns]


##Dictionary

In [5]:
def isNotNull(value):
    return value is not None and len(value)>0

dict_pos = []
dict_neg = []
f = open('negative-words.txt','r', encoding='gbk')
for line in f:
    t= line.strip().lower();
    if (isNotNull(t)):
        dict_neg.append(t)
f.close()

f = open('positive-words.txt','r')
for line in f:
    t = line.strip().lower();
    if (isNotNull(t)):
        dict_pos.append(t)
f.close()
print('sentiment dictionary complete')     

sentiment dictionary complete


## sentiment analysis

In [6]:
import nltk
nltk.download('punkt')
analysis_for_pos = []
for i in range(len(data_needed['collect'].values)):
    tokens = nltk.word_tokenize(data_needed['collect'].values[i])
    neg_cnt = 0
    pos_cnt = 0
    for neg in dict_neg:
        if (neg in tokens):
            neg_cnt = neg_cnt +1
    for pos in dict_pos:
        if (pos in tokens):
            pos_cnt = pos_cnt +1
    analysis_for_pos.append(pos_cnt - neg_cnt)     
data_needed['analysis_for_pos'] = analysis_for_pos

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
data_needed.to_csv('result.csv')
print(data_needed)

                                                    Q30  ... analysis_for_pos
0                                                   NaN  ...               -1
6     picked the products without really think about...  ...                1
18                                                  NaN  ...                0
25    smooth response to controls and movement, ease...  ...                3
53                                                  NaN  ...                1
...                                                 ...  ...              ...
2048  quite peaceful but almost a bit too quiet. Fel...  ...                2
2070                                                NaN  ...                1
2077  I was looking at all the available items to se...  ...                3
2101                                                NaN  ...                0
2114  Hoping the hand-helds would work quickly when ...  ...                1

[102 rows x 6 columns]


##Vlookup

In [21]:
id2score = {}
for i, value in enumerate(data_needed['id']):
  id2score[value] = data_needed['analysis_for_pos'].iloc[i]
last_col = [id2score[value] if value in id2score.keys() else None for i, value in enumerate(data['Part.ID1'])]

In [29]:
data['Sentiment Score'] = last_col

In [30]:
data.to_csv('data.csv')